In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
from os import walk
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn import preprocessing
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers, losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle
import h5py

In [ ]:
loc = 'turbine' # location
modelID = 3
# (0 for SCADA only, 1 for SCADA+Acc17, 2 for SCADA+Acc38, 3 for SCADA+Acc77, 
# 4 for SCADA+Acc17&38, 5 for SCADA+Acc17&38&77 
include_wave = 'no'
duration = '24M'

durations = ['3M','6M','9M','12M','15M','18M','21M','24M']
train_end_dates = ['2018-03-31 23:50:00+00:00', '2018-06-30 23:50:00+00:00', '2018-09-30 23:50:00+00:00', 
                  '2018-12-31 23:50:00+00:00', '2019-03-31 23:50:00+00:00', '2019-06-30 23:50:00+00:00',
                  '2019-09-30 23:50:00+00:00', '2019-12-31 23:50:00+00:00']
train_end_date = train_end_dates[durations.index(duration)]

# Laod train data 
train_input = pd.read_pickle('DATA/train_input')
train_output = pd.read_pickle('DATA/train_output')
index = train_input.columns

train_input = train_input.loc['2018-01-01 00:00:00+00:00':train_end_date]
train_output= train_output.loc['2018-01-01 00:00:00+00:00':train_end_date]
# print(train_input.shape)

# Normlaization of input data
# Data normalization according to training dataset/ model
filehandler = open('Weights/Norm', 'rb') 
std_scaler = pickle.load(filehandler)
inputn = pd.DataFrame(std_scaler.transform(train_input), columns=train_input.columns) 
# inputn is still a daraframe with numeric index
outputn = train_output/10**5  #  change of units, outputn is still a daraframe with time index

In [ ]:
index1 = pd.core.indexes.base.Index([]) # create a blank index array
if include_wave == 'yes': 
    index1 = index1.append(index[0:3])
if modelID == 1: # Acc17
    index1 = index1.append(index[[3,4,9,10,15,16]])
if modelID == 2: # Acc38
    index1 = index1.append(index[[5,6,11,12,17,18]])
if modelID == 3: # Acc77
    index1 = index1.append(index[[7,8,13,14,19,20]])
if modelID == 4: # Acc17&38
    index1 = index1.append(index[[3,4,5,6,9,10,11,12,15,16,17,18]])   
if modelID == 5: # Acc17&38&77
    index1 = index1.append(index[3:21])

index1 = index1.append(index[21:]) # SCADA
X = inputn[index1].values
Y = outputn.values

print(X.shape)
print(Y.shape)

BNN Architecture and Training

In [ ]:
def NLL(y, distr): 
  return -distr.log_prob(y) 

def normal_sp(params): 
  return tfp.distributions.Normal(loc=params[:,0:2], scale=1e-3 
                                  + tf.math.softplus(0.05 * params[:,2:4]))# both parameters are learnable

kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (X.shape[0] * 1.0)
bias_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (X.shape[0] * 1.0)

inputs = tf.keras.layers.Input(shape=(X.shape[1],))

hidden = tfp.layers.DenseFlipout(31,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(inputs)
hidden = tfp.layers.DenseFlipout(64,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
hidden = tfp.layers.DenseFlipout(32,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
params = tfp.layers.DenseFlipout(4,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn)(hidden)
dist = tfp.layers.DistributionLambda(normal_sp)(params)


model = Model(inputs=inputs, outputs=dist)
model.compile(Adam(learning_rate=0.0003), loss=NLL) 

model_params = Model(inputs=inputs, outputs=params)
model.summary()

#train the model
epoch = 1000
batch_size = 1024
history1 = model.fit(X, Y, batch_size=batch_size, epochs=epoch, verbose=1, validation_split = 0.2); # To save weight history, add callback1 and/or callback2


DNN Architecture and Training


In [ ]:
in_dim = X.shape[1]
out_dim = Y.shape[1]
VSnet = Sequential()
VSnet.add(Dense(64, input_dim=in_dim, activation="relu"))
VSnet.add(Dense(128, activation="relu"))
VSnet.add(Dense(64, activation="relu"))
VSnet.add(Dense(out_dim, activation="relu"))

loss = losses.mean_absolute_error
optimizer = optimizers.Adamax(learning_rate = 0.001)
VSnet.compile(loss = loss, optimizer = optimizer)
 
VSnet.summary()

history2 = VSnet.fit(X, Y, validation_split=0.2, epochs=1000, batch_size=32, verbose=1)

In [ ]:
cm = 1/2.54  # centimeters in inches
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 9
# fig, ax = plt.subplots(1, figsize=(7.5*cm, 6*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
fig, ax = plt.subplots(2,1, figsize=(8.5*cm, 7.5*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
ax[0].plot(history2.history['loss'][0:700],'k',linewidth=1)
ax[0].plot(history2.history['val_loss'][0:700],'r',linewidth=1, ls='--')
ax[0].legend(['DNN (Training loss)', 'DNN (Validation loss)'], loc='upper right', ncol=1)
ax[0].set_ylabel('MAE')
ax[0].set_ylim([0.071,0.151])
ax[0].set_xticklabels([])
# ax2 = ax1.twinx()
ax[1].plot(history1.history['loss'][0:700],'k',linewidth=1)
ax[1].plot(history1.history['val_loss'][0:700],'r',linewidth=1, ls='--')
ax[1].legend(['BNN (Training loss)', 'BNN (Validation loss)', 'DNN (Training loss)', 'DNN (Validation loss)'], loc='upper right')

ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('-ELBO')
ax[1].set_ylim([-1,5])

# 
# 
# ax.set_title('')
# ax.set_yscale('log')
# ax.legend()
# fig.show()
plt.subplots_adjust(left=0.15, right=.98, top=0.98, bottom=0.15, hspace = 0.1, wspace=0.15)
fig.savefig('Figures/Overfit.pdf')